In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet

In [2]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        self.classes = self.data['hotel_id'].unique().tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data.iloc[idx, 1]), str(self.data.iloc[idx, 0]))
        image = Image.open(img_name).convert("RGB")
        label = self.classes.index(self.data.iloc[idx, 1])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
train_dataset = CustomDataset('train.csv', 'final/train_images', transform=data_transforms)
best_checkpoint_path = os.path.join("checkpoints", 'efficientnet-b0_best_checkpoint.pth')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
best_model = EfficientNet.from_pretrained('efficientnet-b0')
num_features = best_model._fc.in_features 
best_model._fc = nn.Sequential(
    nn.Linear(num_features, len(train_dataset.classes)),
    nn.Dropout(0.5)  # Adding dropout layer for regularization
)
best_model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
# Load the state_dict, matching the keys correctly
checkpoint = torch.load(best_checkpoint_path)
state_dict = checkpoint['model_state_dict']

# Update the state_dict keys to match the model's current state_dict
state_dict = {k.replace("_fc.0", "_fc.0.weight").replace("_fc.2", "_fc.0.bias"): v for k, v in state_dict.items()}

# Load the modified state_dict to the model
best_model.load_state_dict(state_dict)

In [ ]:
# Initialize optimizer and scheduler
optimizer = optim.Adam(best_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)


criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Evaluate on the validation dataset
val_dataset = CustomDataset('validation.csv', 'final/validation_images', transform=data_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

best_model.train()  # Set the model to train mode

num_epochs = 20
best_val_accuracy = 0.0  # Track the best validation accuracy

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_accuracy = 100 * correct / total
    average_loss = running_loss / len(train_dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Training Loss: {average_loss:.4f}")
    print(f"Training Accuracy: {train_accuracy:.2f}%")

    # Adjust learning rate using the scheduler
    scheduler.step(train_accuracy)

    # Evaluate on the validation dataset
    best_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = best_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")
    print()

    # Save checkpoint
    checkpoint_path = os.path.join("checkpoints_next20", f"efficientnet-b0_checkpoint_{epoch+1}.pth")
    torch.save(best_model.state_dict(), checkpoint_path)

    # Save best checkpoint
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_checkpoint_path = os.path.join("checkpoints_next20", "efficientnet-b0_best_checkpoint.pth")
        torch.save(best_model.state_dict(), best_checkpoint_path)

Epoch 1/20:
Training Loss: 0.2504
Training Accuracy: 0.81%
Validation Accuracy: 3.04%

Epoch 2/20:
Training Loss: 0.1895
Training Accuracy: 13.01%
Validation Accuracy: 27.98%

Epoch 3/20:
Training Loss: 0.0822
Training Accuracy: 52.02%
Validation Accuracy: 49.87%

Epoch 4/20:
Training Loss: 0.0313
Training Accuracy: 77.68%
Validation Accuracy: 55.47%

Epoch 5/20:
Training Loss: 0.0127
Training Accuracy: 89.37%
Validation Accuracy: 55.85%

Epoch 6/20:
Training Loss: 0.0091
Training Accuracy: 91.85%
Validation Accuracy: 56.53%

Epoch 7/20:
Training Loss: 0.0075
Training Accuracy: 93.19%
Validation Accuracy: 56.81%

Epoch 8/20:
Training Loss: 0.0061
Training Accuracy: 94.54%
Validation Accuracy: 57.46%

Epoch 9/20:
Training Loss: 0.0060
Training Accuracy: 94.58%
Validation Accuracy: 58.31%

Epoch 10/20:
Training Loss: 0.0049
Training Accuracy: 95.58%
Validation Accuracy: 56.97%

Epoch 11/20:
Training Loss: 0.0050
Training Accuracy: 95.30%
Validation Accuracy: 56.83%

Epoch 12/20:
Training